 # Libraries Installation

### OS Issues

* EPANET Simulator under wntr doesn't work in MacOS, Linux but does in Windows
* Windows has (.dll) precompiled but MacOS needs it to be compiled (.dylib)
* Working on compiling .dylib and .so for MacOS and Linux

In [23]:
# MacOS specific .dylib 
from pathlib import Path
import os
import platform

system_os = platform.system()

# Get current script directory
base_path = Path().resolve()

if system_os == "Darwin":  # macOS
    epanet_lib_path = base_path / "lib" / "libepanet.dylib"
    os.environ["EPANET_PATH"] = str(epanet_lib_path)

elif system_os == "Linux":
    epanet_lib_path = base_path / "lib" / "libepanet.so"
    os.environ["EPANET_PATH"] = str(epanet_lib_path)

elif system_os == "Windows":
    print("Windows detected. No need to set EPANET_PATH.")

else:
    raise EnvironmentError(f"Unsupported OS: {system_os}")



In [24]:
# Upgrade pip and install packages from requirements.txt (cross-platform safe)
%pip install --upgrade pip
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Genetic Algorithm

## Setting up Manual Functions

### Importing Libraries

In [25]:
import wntr
import numpy as np
from datetime import timedelta
import pandas as pd
import random
import copy

### Auxiliary Functions

In [26]:
# --- RESHAPE INDIVIDUAL TO SCHEDULE ---

def reshape_individual(individual, wntk):

    # Reshape the individual to a 2D array
    schedule = np.array(individual).reshape(wntk.describe(level=1)['Links']['Pumps'], (int(wntk.options.time.duration / wntk.options.time.hydraulic_timestep) + 1)).tolist()

    return schedule

In [27]:
# --- SCHEDULE ADDITION ---

def add_schedule(schedule, wntk):
    for i in range(wntk.describe(level=1)['Links']['Pumps']):
        for j in range(int(wntk.options.time.duration / wntk.options.time.hydraulic_timestep) + 1):
            pump = wntk.get_link(wntk.pump_name_list[i])
            condition = wntr.network.controls.SimTimeCondition(wntk, '=', str(timedelta(hours=j)))
            action = wntr.network.controls.ControlAction(pump, 'status', schedule[i][j])
            control = wntr.network.controls.Control(condition, action, name=f'Control_pump{i}_time{j}')
            wntk.add_control(f"Control Pump ID : {i}, Hour : {j}", control)

In [28]:
# --- ADDED SCHEDULE REMOVAL {EXCLUSIVELY THE ONES FROM ADD SCHEDULE}---

def remove_schedule(wntk):
    for i in range(wntk.describe(level=1)['Links']['Pumps']):
        for j in range(int(wntk.options.time.duration / wntk.options.time.hydraulic_timestep) + 1):
            wntk.remove_control(f"Control Pump ID : {i}, Hour : {j}")

In [29]:
# --- COST OBJECTIVE FUNCTION ---
def cost_objective(wntk):
    # Running Epanet simulation
    simulation = wntr.sim.EpanetSimulator(wntk)
    results = simulation.run_sim()

    # Obtaining cost objective
    base_cost = 5 # Base cost per hour
    cost_pattern = np.array([0.7]*6 + [1.2]*12 + [1]*7).T # Len must be equal to the number of time steps + 1 (25 in this case)
    energy_array = results.link['headloss'][wntk.pump_name_list].to_numpy().T  
    cost_array = cost_pattern * energy_array              
    total_cost = -1 * np.sum(np.sum(cost_array)) * base_cost # Headloss is -ve in Pump, Hence the -1 to obtain the cost
    
    return total_cost

In [30]:
# --- DEMAND OBJECTIVE FUNCTION ---
def demand_objective(wntk):
    # NOTE : The required_pressure, minimum_pressure, pressure_exponent can be applied separately for each node (refer to the WNTR documentation hydraulics.rst)
    # For sake of simplicity, we are using the same values for all nodes

    # Setting up Pressure Driven Demand model
    wntk.options.hydraulic.demand_model = 'PDD'
    wntk.options.hydraulic.required_pressure = 50
    
    # Running WNTR simulation
    simulation = wntr.sim.WNTRSimulator(wntk)
    results = simulation.run_sim()

    # Obtaining demand objective
    expected_demand = wntr.metrics.expected_demand(wntk)
    demand = results.node['demand'].loc[:,wntk.junction_name_list]    
    wsa = wntr.metrics.water_service_availability(expected_demand, demand)
    wsd = 1 - wsa.where(pd.isna(wsa), wsa) # water service deficit
    total_demand = np.nansum(wsd.to_numpy())

    return total_demand

## Setting up DEAP framework

### Setting up Libraries

In [31]:
# Necessary Libraries for DEAP and Torch
from deap import base, tools, creator, algorithms

# --- CONFIGURATION ---
wn = wntr.network.WaterNetworkModel('Network Files/Net3.inp')
NUM_PUMPS = wn.describe(level=1)['Links']['Pumps']
NUM_TIMESTAMPS = (int(wn.options.time.duration / wn.options.time.hydraulic_timestep) + 1)
CHROM_SIZE = NUM_PUMPS * NUM_TIMESTAMPS  # No of Pumps * No of Timestamps

### Evaluation Function

In [32]:
# --- EVALUATION FUNCTION ---

def evaluation(individual, wntk = wn):

    wntk_copy = copy.deepcopy(wntk)

    # Reshape the individual to a schedule
    schedule = reshape_individual(individual, wntk_copy)

    # Add the schedule to the WNTR model
    add_schedule(schedule, wntk_copy)

    # Set Demand Analysis Model (PDA)
    wntk_copy.options.hydraulic.demand_model = 'PDD'
    wntk_copy.options.hydraulic.required_pressure = 150

    # Get Cost Objective - NOTE !!! Price pattern is defined in cost objective function and can be changed inside the cost objective function if necessary
    cost = cost_objective(wntk_copy)

    # Get Demand Objective
    demand = demand_objective(wntk_copy)

    # Reset the simulation
    remove_schedule(wntk_copy)
    wntk_copy.reset_initial_values()

    return cost , demand

In [ ]:
# --- CREATE TYPES ---

# Assigning Mult-Class Fitness Weigths
creator.create("FitnessMulti", base.Fitness, weights=(-1.0,-1000.0,))

# Defining Individual
creator.create("Individual", list, fitness = creator.FitnessMulti)

toolbox = base.Toolbox()

# --- CREATE TOOLS ---

# Define Binary Genes
toolbox.register("attr_bool", random.randint, 0, 1)

# Define Individual (List of Binary Values)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=CHROM_SIZE)

# Define Random Population
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register the Fitness function
toolbox.register("evaluate", evaluation)

# Define Selection Methodology
toolbox.register("select", tools.selNSGA2)

# Define Crossover Methodology
toolbox.register("mate", tools.cxTwoPoint)

# Define Mutation Methodology
toolbox.register("mutate", tools.mutFlipBit, indpb = 0.1)

# Set Hall of Fame
hof = tools.HallOfFame(10)

In [ ]:
# --- GENETIC ALGO LOOP---

def run_ga(pop_size = 40, generations = 5, cx_prob = 0.7, mut_prob = 0.3):
    #Create
    population = toolbox.population(n = pop_size)

    # Multi-Objective Statistics
    stats = tools.Statistics(lambda ind: ind.fitness.values)

    # Register statistics for each objective separately
    stats.register("avg_cost", lambda fits: np.mean([f[0] for f in fits]))
    stats.register("min_cost", lambda fits: np.min([f[0] for f in fits]))
    stats.register("max_cost", lambda fits: np.max([f[0] for f in fits]))

    stats.register("avg_demand", lambda fits: np.mean([f[1] for f in fits]))
    stats.register("min_demand", lambda fits: np.min([f[1] for f in fits]))
    stats.register("max_demand", lambda fits: np.max([f[1] for f in fits]))

    # Run the Genetic Algorithm
    population, logbook = algorithms.eaMuPlusLambda(
        population=population,
        toolbox=toolbox,
        mu = round(pop_size * 0.7),
        lambda_= round(pop_size * 0.5),
        cxpb=cx_prob,
        mutpb=mut_prob,
        ngen=generations,
        stats=stats,
        halloffame=hof,
        verbose=True,
    )
    
    best_ind = hof[0]
    print("Best Individual : ", best_ind)
    print("Fitness : ", best_ind.fitness.values)

    return best_ind, logbook, population, hof

best_sol, logbook, population, hof = run_ga()